# cryoQuery

In [1]:
# |default_exp cryo_query

In [2]:
# | export
import jupyter_black
import os
import polars as pl
import re
import cryo

from dataclasses import dataclass

pl.Config.set_fmt_str_lengths(200)
pl.Config.set_fmt_float("full")
jupyter_black.load()

In [3]:
cd ../

/home/evan/Documents/ethereum_block_explorer


/home/evan/Documents/ethereum_block_explorer/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
# | export
@dataclass
class cryoQuery:
    """
    `cryoQuery` is used to query data from cryo
    """

    raw_data_path: str = "data/raw"
    rpc: str = "https://eth.merkle.io"

    def _create_data_filepaths(self):
        """
        Creates folders for storing raw data from cryo.
        """
        if not os.path.exists(self.raw_data_path):
            os.makedirs(self.raw_data_path)
            print("Data folder created.")
        else:
            print("Data folder already exists.")

    def query_blocks_txs(
        self,
        n_error_threshold: int = 1,
        retry_threshold: int = 5,
        block_range: list[str] = ["latest"],
        name: str = None,
    ):
        """
        Fetches block and transaction data. Attempts retries up to 'retry_threshold'. If errors exceed 'n_error_threshold'.

        :param n_error_threshold: The number of allowed errors before retrying the query.
        :param retry_threshold: The maximum number of retries for the query.
        :param block_range: The range of blocks to query. Defaults to ["latest"].
        """
        self._create_data_filepaths()
        n_errored = n_error_threshold + 1
        retry_count = 0

        # make cryo query
        while retry_count < retry_threshold:
            print(f"Retry count: {retry_count}")
            retry_count += 1
            if n_error_threshold < n_errored:
                output: dict[str] = cryo.freeze(
                    "blocks_and_transactions",
                    blocks=block_range,
                    hex=True,
                    rpc=self.rpc,
                    no_verbose=False,  # this doesn't seem to have any effect
                    output_dir=f"data/raw_{name}",
                    subdirs=["datatype"],
                    include_columns=["n_rlp_bytes"],
                    exclude_columns=["input", "value"],
                    # compression=["zstd"],
                    compression=["lz4"],  # bug, can't use zstd in cryo 0.3.0
                )

                n_errored = output["n_errored"]
                print(f"Number of errors: {n_errored}")
            if n_errored == 0:
                print(f"{n_errored} == 0. Done!")
                break


# | export
@dataclass
class cryoTransform:
    """
    cryoTransform extends the underlying transactions and blocks dataset with extra columns.
    """

    def extend_txs_blocks(
        self,
        txs: pl.LazyFrame,
        blocks: pl.LazyFrame,
        mempool: pl.LazyFrame | None = None,
    ) -> pl.LazyFrame:
        """
        Combines transaction, block, and optionally flashbots mempool data into a single LazyFrame.

        Uses pattern matching to handle the optional mempool data.
        If mempool data is provided, it is joined with the transactions and blocks data.
        If not, only transactions and blocks are joined.

        Preprocessing:
        - add block percentile and ticks
        - convert gas to gwei
        - convert bytes to kilobytes


        Parameters:
        - txs (pl.LazyFrame): LazyFrame containing transaction data.
        - blocks (pl.LazyFrame): LazyFrame containing block data.
        - mempool (pl.LazyFrame, optional): LazyFrame containing mempool data. Default is None.

        Returns:
        - pl.LazyFrame: A unified LazyFrame with enriched transaction data.
        """

        # Use pattern matching to handle the presence or absence of mempool data
        match mempool:
            case _ if isinstance(mempool, pl.LazyFrame):
                # Join transactions with blocks and mempool data if mempool is provided
                combined_df = txs.join(
                    blocks, on="block_number", how="left", suffix="_block"
                ).join(
                    mempool,
                    right_on="hash",
                    left_on="transaction_hash",
                    how="left",
                    suffix="_mempool",
                )

            case None:
                # Join only transactions with blocks if mempool is not provided
                combined_df = txs.join(
                    blocks, on="block_number", how="left", suffix="_block"
                )

        agg_df: pl.LazyFrame = combined_df.group_by("block_number").agg(
            [
                pl.col("transaction_index").max().alias(
                    "transaction_index_max"),
                pl.col("n_rlp_bytes").sum().alias("block_encoded_bytes"),
                pl.col("n_input_bytes").sum().alias("block_calldata_bytes"),
            ]
        )

        return (
            combined_df.join(agg_df, on="block_number", how="left")
            .with_columns(
                [
                    # Calculate the transaction gas cost
                    (pl.col("gas_used") * pl.col("gas_price") / 10**18).alias(
                        "tx_gas_cost"
                    ),
                    # Convert epoch timestamp to datetime
                    pl.from_epoch("timestamp").alias("block_datetime"),
                    # Calculate the gas price premium over the base fee per gas
                    (pl.col("gas_price") / pl.col("base_fee_per_gas")).alias(
                        "block_gas_premium"
                    ),
                    (pl.col("base_fee_per_gas").rolling_mean(window_size=7200)).alias(
                        "avg_base_fee_daily"
                    ),
                    (pl.col("base_fee_per_gas").rolling_mean(window_size=300)).alias(
                        "avg_base_fee_hourly"
                    ),
                    (pl.col("base_fee_per_gas").rolling_mean(window_size=5)).alias(
                        "avg_base_fee_minute"
                    ),
                ]
            )
            .with_columns(
                # Calculate the transaction index percentile within its block
                (
                    pl.col("transaction_index") / \
                    pl.col("transaction_index_max") * 100
                ).alias("blockspace_percentile")
            )
            .with_columns(
                # Round the block space percentile for easier interpretation
                (pl.col("blockspace_percentile").round()).alias(
                    "rounded_blockspace_percentile"
                )
            )
            # unit conversions
            .with_columns(
                # convert gas to gwei
                (pl.col("gas_price") / 10**9).alias("gas_price_gwei"),
                (pl.col("max_priority_fee_per_gas") / 10**9).alias(
                    "max_priority_fee_per_gas_gwei"
                ),
                (pl.col("max_fee_per_gas") / 10**9).alias("max_fee_per_gas_gwei"),
                (pl.col("base_fee_per_gas") / 10 **
                 9).alias("base_fee_per_gas_gwei"),
                (pl.col("avg_base_fee_daily") / 10**9).alias(
                    "avg_base_fee_daily_gwei"
                ),
                (pl.col("avg_base_fee_hourly") / 10**9).alias(
                    "avg_base_fee_hourly_gwei"
                ),
                (pl.col("avg_base_fee_minute") / 10**9).alias(
                    "avg_base_fee_minute_gwei"
                ),
                # convert bytes to kilobytes
                (pl.col("block_encoded_bytes") / 10 **
                 3).alias("block_encoded_kbytes"),
                (pl.col("block_calldata_bytes") / 10**3).alias(
                    "block_calldata_kbytes"
                ),
            )
            .drop(
                "gas_price",
                "max_priority_fee_per_gas",
                "max_fee_per_gas",
                "base_fee_per_gas",
                "block_encoded_bytes",
                "block_calldata_bytes",
            )
            .fill_nan(0)  # Fill NaN values with 0
            .unique()  # Ensure all rows are unique
        )

    # 1. read the files in the raw data from block number partitions, sync them together, perform a transformation, and then save into a new folder.
    def read_filenames(self, directory) -> list:
        try:
            return sorted(
                [
                    f
                    for f in os.listdir(directory)
                    if os.path.isfile(os.path.join(directory, f))
                ]
            )
        except FileNotFoundError:
            return []

    def extract_block_batch_index(self, filename) -> list[tuple]:
        match = re.search(r"to_(\d+)", filename)
        return int(match.group(1)) if match else None

    def sync_filenames(self, directory_a: str, directory_b: str):
        # Read filenames from both directories
        transactions_filenames = self.read_filenames(directory_a)
        blocks_filenames = self.read_filenames(directory_b)

        # Extract numbers and create mappings
        transactions_mapping = {
            self.extract_block_batch_index(name): name
            for name in transactions_filenames
        }
        blocks_mapping = {
            self.extract_block_batch_index(name): name for name in blocks_filenames
        }

        # Find common keys
        common_keys = set(transactions_mapping.keys()).intersection(
            blocks_mapping.keys()
        )

        # Creating synced files dictionary
        synced_files = {
            directory_a: [transactions_mapping[key] for key in common_keys],
            directory_b: [blocks_mapping[key] for key in common_keys],
        }

        return synced_files

In [6]:
# sample usage
cryo_query = cryoQuery(
    rpc="https://rpc.merkle.io/42161/sk_mbs_52643548f289817e03f67d550224141c"
)

cryo_query.query_blocks_txs(block_range=["174061989:174161989"])

Data folder already exists.
Retry count: 0
cryo parameters
───────────────
- version: 0.3.2
- data: 
    - datatypes: blocks, transactions
    - blocks: n=100,000 min=174,061,989 max=174,161,988 align=no reorg_buffer=0
- source: 
    - network: arbitrum
    - rpc url: https://rpc.merkle.io/42161/sk_mbs_52643548f289817e03f67d550224141c
    - max requests per second: unlimited
    - max concurrent requests: unlimited
    - max concurrent chunks: 4
- output: 
    - chunk size: 1,000
    - chunks to collect: 100 / 100
    - output format: parquet
    - output dir: /home/evan/Documents/ethereum_block_explorer/data/raw
    - report file: $OUTPUT_DIR/.cryo/reports/2024-01-25_17-08-53.350060.json


schema for blocks
─────────────────
- block_number: uint32
- block_hash: hex
- timestamp: uint32
- author: hex
- gas_used: uint64
- extra_data: hex
- base_fee_per_gas: uint64
- chain_id: uint64

sorting blocks by: block_number

other available columns: parent_hash, state_root, transactions_root, rec

In [ ]:
data = pl.read_parquet("data/raw/blocks/*.parquet")

In [ ]:
data

block_hash,author,block_number,gas_used,extra_data,timestamp,base_fee_per_gas,chain_id
str,str,u32,u64,str,u32,u64,u64
"""0xbc8ec21c3ef5714908b083bf8f415e77e3036ed3f4e8823c0095d4f038260612""","""0xa4b000000000000000000073657175656e636572""",174161989,1900624,"""0x570571e8c3aac708dd897a3d64a118c3daf365c88bbc5669ed27ad8f5609d476""",1706218447,100000000,42161
